In [ ]:
#Seas DBG deformation mapping code
#Author J.F.Zimmerman
#Updated 5/13/2022
#Takes images of feducial markers placed on the surface of a ventricle, and performs DPIV
#Runs on the OPENPIV framework

from openpiv import tools, scaling, pyprocess, validation
import openpiv
import numpy as np
import os
from matplotlib.pyplot import *
from openpiv import process
from openpiv import tools
from mpl_toolkits.mplot3d.axes3d import Axes3D
%matplotlib inline
import glob
import PIVfilters
import scipy.ndimage as ndimage
import scipy.ndimage.filters as filters
import scipy.interpolate
import PIL
import skimage


import pyximport
pyximport.install()


In [ ]:
def sortGlob(s):
    #Assumes files have a 4 digit ID number, e.g. "File0001.tif"
    return int(os.path.basename(s)[-8:-4])

def getMag(u,v):
    return np.sqrt(u*u+v*v)

def AdjOutliers( u2, v2, threshold=1.0, kernel_size=3):
    u3 = u2.copy()
    v3 = v2.copy()
    mag = getMag(u3,v3)
    mag = mag/np.average(mag)
    std = ndimage.laplace(1-mag)
    outliers = np.where(std>threshold)
    
    #If it contains outliers, perform filtering
    if outliers[0].size>0:
        umean = ndimage.generic_filter(u3, np.nanmean, size=3, mode='constant', cval=np.NaN)
        vmean = ndimage.generic_filter(v3, np.nanmean, size=3, mode='constant', cval=np.NaN)
        zerogrid = np.zeros(u3.shape)
        zerogrid[outliers] = 1
        onesgrid = np.ones(u3.shape)
        onesgrid[outliers] = 0
        u3 = zerogrid*umean+onesgrid*u3
        v3 = zerogrid*vmean+onesgrid*v3
        #Report the number of outliers filtered
        print(str(outliers[0].size)+' outliers filtered.')
    return u3,v3

def gen_mask(frame_a,mask):
    #Takes a 2D image from 0-255, and returns processed mask, which removes the background
    val = skimage.filters.threshold_li(frame_a)
    invalid = frame_a<val
    valid = ~invalid
    valid = ndimage.morphology.binary_fill_holes(valid)
    valid = ndimage.morphology.binary_erosion(valid)

    valid = ndimage.morphology.binary_dilation(valid)
    
    #Resize grid to match original  mask size
    im = PIL.Image.fromarray(valid.astype(np.uint8)).resize((mask.shape[1],mask.shape[0]))
    valid = np.array(im)
    
    
    #Invert
    invalid = ~valid
    
    #Determine the overlap
    xor = ~invalid-~invalid*mask
    valid = ~xor
        
    return valid



In [ ]:
#Takes raw flourscent particle images, and converts them into ventricle deformation maps
#Assumes that the initial frame occurs during diastole
#--------------- Inputs ----------------------
#File Folders
rootpath = 'Data\\Raw\\'
savepath = 'Data\\Analyzed\\'

#PIV Values
FPS = 30
winsize = 60 # pixels
searchsize = 100 # pixels, search in image B
overlap = 15 # pixels
scale = 0.013236 #mm/pixel - Pixel resolution of the input camera
threshVal = 1.05 #Background thresholding, How much of the image to mask, Default = 1.2

#Graphing Figures Inputs
figheight = 6 #Figure height, Size of the output images
s = 3 #Image Smoothing
vlowlim,vuplim = 0,0.1 #


#Vector Mapping
showvectorfields = False
skip=5 #density of vector arrows, higher values reduce the number, must be an interger
vwidth, vscale = 0.005,1 #width and scale of vector arrows in graphing

#--------------- Functional Code ----------------------
dt = 1.0/FPS
root = glob.glob(rootpath+'*')
vlowlim,vuplim = vlowlim*100,vuplim*100 # mm to um/mm normalization

#Setting up Plot 
cms = matplotlib.cm.CMRmap
normv = matplotlib.colors.Normalize()
normv.autoscale(np.array([vlowlim,vuplim]))
vsm = matplotlib.cm.ScalarMappable(cmap=cms,norm=normv)
vsm.set_array([])
matplotlib.rcParams['font.sans-serif'] = "Arial"
cbar_labels = np.linspace(vlowlim,vuplim,6)


for angle in root:
    print('Searching in..')
    print(os.path.basename(angle))
    subfolders = glob.glob(angle+'\\*')
    
    if os.path.isdir(savepath+os.path.basename(angle)):
        print('SaveAngle-Confirm')
    else:
        os.mkdir(savepath+os.path.basename(angle))
    
    for cant in subfolders:
        print(cant)
        files = glob.glob(cant+'\\*.tif')
        files.sort(key=sortGlob)
        print(len(files))
        
        
        if os.path.isdir(savepath+os.path.basename(angle)+'\\'+os.path.basename(cant)):
            print('SaveCant-Confirm')
        else:
            os.mkdir(savepath+os.path.basename(angle)+'\\'+os.path.basename(cant))
        
        for i in range (0,len(files)-1):

            if i ==0:
                frame_a  = tools.imread( files[i] )

            
            frame_b  = tools.imread( files[i+1] )
            print(files[i+1])
            
            u0, v0, sig2noise = process.extended_search_area_piv( frame_a.astype(np.int32), frame_b.astype(np.int32), window_size=winsize, overlap=overlap, dt=dt, search_area_size=searchsize, sig2noise_method='peak2peak' )
            
            x, y = openpiv.pyprocess.get_coordinates( image_size=frame_a.shape, window_size=winsize, overlap=overlap )
            u1, v1, mask = openpiv.validation.sig2noise_val( u0, v0, sig2noise, threshold = threshVal )
            mask = gen_mask(frame_b,mask)
            imask = ~mask

            u2, v2 = PIVfilters.replace_outliers( u1, v1, method='localmean', max_iter=10, kernel_size=5)
            u3, v3 = AdjOutliers( u2, v2,threshold=2.5, kernel_size=3)
            x, y, u4, v4 =  openpiv.scaling.uniform(x, y, u3, v3, scaling_factor = (1./scale)*100 )
            x,y = x*100,y*100 # mm to um/mm normalization
            print(f'U max: {np.max(u4)}')
            print(f'V max: {np.max(v4)}')
            
            XYUV = np.dstack((x.ravel(),y.ravel(),u4.ravel()*imask.ravel(),v4.ravel()*imask.ravel()))[0,:,:]
                
            xnodes = np.unique(XYUV[:,0]).shape[0]
            ynodes = np.unique(XYUV[:,1]).shape[0]
            AR = xnodes/float(ynodes)
            
            #Generate Mesh Grid Coordinates
            x = np.linspace(np.min(XYUV[:,0]),np.max(XYUV[:,0]),xnodes)
            y = np.linspace(np.min(XYUV[:,1]),np.max(XYUV[:,1]),ynodes)
            dx = x[1]-x[0] ## given in mm
            dy = y[1]-y[0] ## given in mm
            xv,yv = np.meshgrid(x,y)

            #Map Velocities to the grid coordinates
            u = scipy.interpolate.griddata(XYUV[:,0:2],XYUV[:,2],(xv,yv))
            v = scipy.interpolate.griddata(XYUV[:,0:2],XYUV[:,3],(xv,yv))

            #Smooth Grid
            x = np.linspace(np.min(XYUV[:,0]),np.max(XYUV[:,0]),xnodes*s)
            y = np.linspace(np.min(XYUV[:,1]),np.max(XYUV[:,1]),ynodes*s)
            xs,ys = np.meshgrid(x,y)
            
            #Deformation Map Smooth
            M = np.sqrt(XYUV[:,2]**2+XYUV[:,3]**2)*100 #*100 for the mm to um/mm conversion
            print(f'M max: {np.max(M)}')
            gridinterp = scipy.interpolate.griddata(XYUV[:,0:2],M,(xs,ys))
            us = scipy.interpolate.griddata(XYUV[:,0:2],XYUV[:,2],(xs,ys),method='linear')
            vs = scipy.interpolate.griddata(XYUV[:,0:2],XYUV[:,3],(xs,ys),method='linear')
            
            #Make Savepath for Velocity Maps
            if os.path.isdir(savepath+os.path.basename(angle)+'\\'+os.path.basename(cant)):
                print('SaveCant-Confirm')
            else:
                os.mkdir(savepath+os.path.basename(angle)+'\\'+os.path.basename(cant))
            
            #Plot Deformation Map
            fig, ax = subplots(figsize=(figheight*AR,figheight))
            ax.pcolormesh(xs,ys, gridinterp,cmap=cms,vmin=vlowlim,vmax=vuplim)
            if showvectorfields:
                q = ax.quiver(xs[::skip,::skip], ys[::skip,::skip],us[::skip,::skip], vs[::skip,::skip], color='w',clim=(vlowlim,vuplim),width=vwidth,scale=vscale)

            ax.set_xlabel('Y (mm)',size=30,labelpad=20,weight='bold',rotation=180)
            ax.set_ylabel('X (mm)',size=30,labelpad=20,weight='bold')
            for tick in ax.xaxis.get_major_ticks():
                tick.label.set_fontsize(20)
                tick.label.set_rotation(90)
            for tick in ax.yaxis.get_major_ticks():
                tick.label.set_fontsize(20)
                tick.label.set_rotation(90)

            fig.cbar = fig.colorbar(vsm,cmap=cms)
            fig.cbar.set_label((r'Deformation (x10$^{-2}$%)'), rotation=90,fontsize=30,labelpad=30,weight='bold')
            ticklist = fig.cbar.ax.yaxis.get_major_ticks()
            cbar_labels = np.linspace(vlowlim,vuplim,len(ticklist)).astype(int)

            fig.cbar.ax.set_yticklabels(cbar_labels,rotation=90,fontsize=20)
            tight_layout(1.5)
            
            #Save and Close Figure
            saveloc = savepath+os.path.basename(angle)+'\\'+os.path.basename(cant)+'\\'
            fig.savefig(saveloc+'DeformationMap'+str(i)+'.png')
            clf()
            close()
            
            #Save deformation output
            np.savetxt(saveloc+'DeformationXYUV'+str(i)+'.txt',XYUV)